In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import numpy as np
import time
pd.set_option('display.max_columns', 500)
import threading

In [4]:
merged_data_raw = pd.read_csv('merged_data_raw_fe.csv')

In [5]:
merged_data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354794 entries, 0 to 354793
Data columns (total 71 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Unnamed: 0           354794 non-null  int64  
 1   mssv                 354794 non-null  object 
 2   mamh                 354792 non-null  object 
 3   malop                354792 non-null  object 
 4   sotc                 354792 non-null  float64
 5   hocky                354792 non-null  float64
 6   namhoc               354792 non-null  float64
 7   diem_qt              218760 non-null  float64
 8   diem_th              152239 non-null  float64
 9   diem_gk              195942 non-null  float64
 10  diem_ck              340120 non-null  float64
 11  diem_hp              354421 non-null  float64
 12  trangthai            354792 non-null  float64
 13  tinhtrang            354792 non-null  float64
 14  mamh_tt              707 non-null     object 
 15  namsinh          

In [ ]:
merged_data_cleaned = merged_data_raw.dropna(subset=['mssv'])
merged_data_cleaned = merged_data_cleaned.drop(columns=['mssv'])
merged_data_cleaned = merged_data_cleaned.fillna(0)

In [ ]:
label_encoders = {}
for column in merged_data_cleaned.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    try:
        merged_data_cleaned[column] = le.fit_transform(merged_data_cleaned[column])
    except TypeError:
        merged_data_cleaned[column] = le.fit_transform(merged_data_cleaned[column].astype(str))
    label_encoders[column] = le

In [ ]:
X = merged_data_cleaned.drop(columns=['diem_hp'])  # Assuming 'diem_hp' is the target variable
y = merged_data_cleaned['diem_hp']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import pandas as pd
import numpy as np
import threading
import time
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

# Định nghĩa các mô hình
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'Elastic Net Regression': ElasticNet(),
    'LightGBM Regression': LGBMRegressor(),
    'Random Forest Regression': RandomForestRegressor(),
    'Support Vector Regression': SVR()
}

# Giới hạn thời gian huấn luyện là 1 giờ (3600 giây)
training_threshold = 3600  # seconds

# Danh sách các tập dữ liệu đã được tiền xử lý
datasets = ['dataset1.csv', 'dataset2.csv', 'dataset3.csv', 'dataset4.csv', 'dataset5.csv']

# Lặp qua từng tập dữ liệu
for dataset_name in datasets:
    print(f"\nĐang xử lý {dataset_name}")
    # Đọc tập dữ liệu đã được tiền xử lý
    data = pd.read_csv(dataset_name)
    
    # Xác định các cột số và cột phân loại
    numerical = data.select_dtypes(include=['int64', 'float64']).columns.tolist()
    categorical = data.select_dtypes(include=['object']).columns.tolist()
    
    # Định nghĩa biến mục tiêu
    target_variable = 'dtbhk1'  # Thay 'dtbhk1' bằng tên biến mục tiêu thực tế nếu khác
    if target_variable not in data.columns:
        print(f"Biến mục tiêu '{target_variable}' không có trong tập dữ liệu '{dataset_name}'. Bỏ qua tập dữ liệu này.")
        continue
    if target_variable in numerical:
        numerical.remove(target_variable)
    if target_variable in categorical:
        categorical.remove(target_variable)
    
    # Định nghĩa đặc trưng (X) và biến mục tiêu (y)
    X = data.drop(columns=[target_variable])
    y = data[target_variable]
    
    # Chia dữ liệu thành tập huấn luyện và kiểm tra
    try:
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )
    except ValueError as e:
        print(f"Lỗi khi chia dữ liệu cho tập '{dataset_name}': {e}")
        continue
    
    # Vì dữ liệu đã được tiền xử lý, chúng ta có thể tiến hành huấn luyện mô hình trực tiếp

    # Huấn luyện và đánh giá từng mô hình
    for model_name, model in models.items():
        print(f"\nHuấn luyện mô hình: {model_name}")
        
        y_pred = [None]
        training_time = [None]
        training_completed = [False]
        
        def train_model():
            start_time = time.time()
            try:
                model.fit(X_train, y_train)
                y_pred[0] = model.predict(X_test)
                training_time[0] = time.time() - start_time
                training_completed[0] = True
            except Exception as e:
                print(f"Lỗi khi huấn luyện mô hình {model_name}: {e}")
                training_completed[0] = False
        
        # Tạo và bắt đầu thread huấn luyện
        training_thread = threading.Thread(target=train_model)
        training_thread.start()
        training_thread.join(timeout=training_threshold)
        
        if not training_completed[0]:
            print(f"Mô hình {model_name} đã vượt quá thời gian huấn luyện cho phép ({training_threshold} giây) hoặc gặp lỗi.")
            y_pred[0] = np.nan
            training_time[0] = np.nan
        else:
            # Tính toán các độ đo
            mse = mean_squared_error(y_test, y_pred[0])
            rmse = np.sqrt(mse)
            mae = mean_absolute_error(y_test, y_pred[0])
            r_squared = r2_score(y_test, y_pred[0])
            n = len(y_test)
            p = X_test.shape[1]
            if n > p + 1 and p > 0:
                adjusted_r_squared = 1 - (1 - r_squared) * ((n - 1) / (n - p - 1))
            else:
                adjusted_r_squared = r_squared  # Không thể tính Adjusted R-squared
                
            # Hiển thị kết quả
            print(f"MSE: {mse}")
            print(f"RMSE: {rmse}")
            print(f"MAE: {mae}")
            print(f"R-squared: {r_squared}")
            print(f"Adjusted R-squared: {adjusted_r_squared}")
            print(f"Thời gian huấn luyện: {training_time[0]} giây")



Huấn luyện mô hình: Linear Regression
MSE: 0.8765377373435514
RMSE: 0.9362359410659
MAE: 0.6155936399077466
R-squared: 0.8521296724883041
Adjusted R-squared: 0.8519982692774537
Thời gian huấn luyện: 3.036477565765381 giây

Huấn luyện mô hình: Ridge Regression
MSE: 0.8765377389137126
RMSE: 0.9362359419044499
MAE: 0.6155942617987842
R-squared: 0.8521296722234208
Adjusted R-squared: 0.8519982690123351
Thời gian huấn luyện: 0.6021723747253418 giây

Huấn luyện mô hình: Lasso Regression
MSE: 2.3213315882605503
RMSE: 1.5235916737303832
MAE: 1.1037196768322268
R-squared: 0.6083955686156653
Adjusted R-squared: 0.6080475739873106
Thời gian huấn luyện: 0.5960485935211182 giây

Huấn luyện mô hình: Elastic Net Regression
MSE: 2.030488481674836
RMSE: 1.4249520980281534
MAE: 1.0416292679228345
R-squared: 0.6574602735257888
Adjusted R-squared: 0.6571558796648977
Thời gian huấn luyện: 0.789520263671875 giây

Huấn luyện mô hình: LightGBM Regression
MSE: 0.14746461551505285
RMSE: 0.38401121795470095
MAE

Dưới đây là kết quả huấn luyện của các mô hình dưới dạng bảng với format Markdown:

| Mô hình                  | MSE       | RMSE      | MAE       | R-squared | Adjusted R-squared | Thời gian huấn luyện (giây) |
|--------------------------|-----------|-----------|-----------|-----------|--------------------|-----------------------------|
| Linear Regression        | 0.8765    | 0.9362    | 0.6156    | 0.8521    | 0.8520             | 3.0365                      |
| Ridge Regression         | 0.8765    | 0.9362    | 0.6156    | 0.8521    | 0.8520             | 0.6022                      |
| Lasso Regression         | 2.3213    | 1.5236    | 1.1037    | 0.6084    | 0.6080             | 0.5960                      |
| Elastic Net Regression   | 2.0305    | 1.4250    | 1.0416    | 0.6575    | 0.6572             | 0.7895                      |
| LightGBM Regression      | 0.1475    | 0.3840    | 0.2015    | 0.9751    | 0.9751             | 13.0612                     |
| Random Forest Regression | 0.1121    | 0.3348    | 0.1385    | 0.9811    | 0.9811             | 820.1286                    |
| Support Vector Regression| NaN       | NaN       | NaN       | NaN       | NaN                | nan                         |

Nếu bạn cần thêm điều chỉnh nào khác, hãy cho mình biết nhé!